# HRN Inference

This notebook performs HRN inference for the [TryGlasses](https://github.com/secona/tryglasses-web) project.

### Acknowledgements
The HRN model used in this notebook is kindly provided by [ModelScope](https://modelscope.cn/models/iic/cv_HRN_head-reconstruction/summary).

Original Project: [HRN (Hierarchical Representation Network)](https://github.com/youngLBW/HRN)

### Estimated Runtime

The authors report that the full notebook requires approximately one hour to complete when using the free tier of Google Colab GPU. Most of the time goes into installing dependencies, namely `pytorch3d` and `nvdiffrast`.

### Usage Instructions
1. **Prepare Input**: Save or upload your source image to the working directory and rename it to `face.jpg`.
2. **Execution**: Run the cells below sequentially to generate the inference results.
3. **Downloading**: If run on Google Colab, it should automatically download the resulting ZIP file ready for the web app.

In [ ]:
INPUT_IMG_PATH = "face.jpg"

In [ ]:
import os

if not os.path.exists(INPUT_IMG_PATH):
    raise FileNotFoundError(f"Error: '{INPUT_IMG_PATH}' was not found in the current directory. Please upload the image and try again.")

In [ ]:
import cv2
import matplotlib.pyplot as plt

def display_input_img():
  img = cv2.imread(INPUT_IMG_PATH)
  img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  plt.imshow(img_rgb)
  plt.axis('off')
  plt.show()

display_input_img()

# Installation

In [ ]:
!python --version

In [ ]:
!pip install torch torchvision torchaudio
!pip install tensorflow
!pip install "modelscope[cv]" -f https://modelscope.oss-cn-beijing.aliyuncs.com/releases/repo.html
!pip install git+https://github.com/NVlabs/nvdiffrast.git --no-build-isolation
!pip install git+https://github.com/facebookresearch/pytorch3d.git

In [ ]:
import torch
import numpy as np
import os
import cv2
import json
import shutil

from pytorch3d.structures import Meshes
from moviepy.editor import ImageSequenceClip
from modelscope.models.cv.face_reconstruction.utils import write_obj
from modelscope.outputs import OutputKeys
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks

# Model initialization
Initialize the pipeline and download the pretrained weights from modelscope.

In [ ]:
if not hasattr(torch, '_original_load_unpatched'):
    torch._original_load_unpatched = torch.load

def strict_bypass_load(*args, **kwargs):
    if 'weights_only' not in kwargs:
        kwargs['weights_only'] = False

    return torch._original_load_unpatched(*args, **kwargs)

torch.load = strict_bypass_load

head_reconstruction = pipeline(Tasks.head_reconstruction, model='iic/cv_HRN_head-reconstruction', model_revision='v0.1')

# Run Inference

In [ ]:
result = head_reconstruction(INPUT_IMG_PATH)

In [ ]:
model = head_reconstruction.model

# Extract Information

In [ ]:
coeffs = model.pred_coeffs_dict
coeffs.keys()

In [ ]:
rotation = coeffs['angle'].detach().cpu().numpy()
position = coeffs['trans'].detach().cpu().numpy()

print("Rotation (Rad):", rotation)
print("Position:", position)

In [ ]:
opt = model.opt

print("--- Camera Intrinsics ---")

print(f"Focal Length: {opt.focal}")
print(f"Center (cx, cy): {opt.center}")

print("\n--- Camera Extrinsics & Clipping ---")

print(f"Camera Distance: {opt.camera_d}")
print(f"Z Near: {opt.z_near}")
print(f"Z Far:  {opt.z_far}")

fov = 2 * np.arctan(opt.center / model.opt.focal) * 180 / np.pi
print(f"Calculated FOV: {fov:.2f} degrees")

In [ ]:
def extract_preprocessed_input_img(model):
    input_tensor = model.input_img
    input_img_numpy = input_tensor.detach().cpu().numpy()

    if input_img_numpy.ndim == 4:
        input_img_numpy = input_img_numpy[0]

    if input_img_numpy.shape[0] == 3:
        input_img_numpy = np.transpose(input_img_numpy, (1, 2, 0))

    if input_img_numpy.max() <= 1.0:
        input_img_numpy = (input_img_numpy * 255).astype(np.uint8)
    else:
        input_img_numpy = input_img_numpy.astype(np.uint8)

    input_img_rgb = cv2.cvtColor(input_img_numpy, cv2.COLOR_BGR2RGB)

    return input_img_rgb

def view_cropped_input_img(model):
  input_img_rgb = extract_preprocessed_input_img(model)
  original_img_rgb = cv2.cvtColor(cv2.imread(INPUT_IMG_PATH), cv2.COLOR_BGR2RGB)

  fig, ax = plt.subplots(nrows=1, ncols=2)
  ax = ax.flatten()

  ax[0].imshow(input_img_rgb)
  ax[0].axis('off')
  ax[0].set_title("Preprocessed")

  ax[1].imshow(original_img_rgb)
  ax[1].axis('off')
  ax[1].set_title("Original")

  plt.tight_layout()
  plt.show()

view_cropped_input_img(model)

In [ ]:
def verify_projection(model, position, opt):
    img = extract_preprocessed_input_img(model)

    position = position.flatten()
    focal = opt.focal or 1015.0
    center = opt.center or 112.0

    x = (position[0] / position[2]) * focal + center
    y = (position[1] / position[2]) * focal + center

    cv2.circle(img, (int(x), int(y)), 5, (0, 255, 0), -1)

    plt.imshow(img)
    plt.title(f"Projected Center\n(X={x:.1f}, Y={y:.1f})")
    plt.axis('off')
    plt.show()

verify_projection(model, position, opt)

# Save Results

In [ ]:

def save_results(result, model, save_root):
    os.makedirs(save_root, exist_ok=True)

    mesh = result[OutputKeys.OUTPUT]['mesh']
    write_obj(os.path.join(save_root, 'HRN_result.obj'), mesh)

    img = extract_preprocessed_input_img(model)
    cv2.imwrite(os.path.join(save_root, 'HRN_preprocessed.jpg'), img)

    print(f'Output written to {os.path.abspath(save_root)}')

save_results(result, model, './HRN_export')
!zip -r HRN_export.zip HRN_export

In [ ]:
def save_reconstruction_data(filename, model, result, coeffs, opt, image_path):
    vertices = model.pred_vertex.detach().cpu().numpy()[0] # [N, 3]
    faces = model.headmodel.face_buf.detach().cpu().numpy() # [M, 3]
    mesh = result['output']['mesh']

    texture_map = result['output_img']
    if hasattr(texture_map, 'detach'):
        texture_map = texture_map.detach().cpu().numpy()

    trans = coeffs['trans'].detach().cpu().numpy().flatten()
    angle = coeffs['angle'].detach().cpu().numpy().flatten()

    camera_info = {
        'focal': opt.focal,
        'center': opt.center,
        'camera_d': opt.camera_d,
        'z_near': opt.z_near,
        'z_far': opt.z_far
    }

    np.savez_compressed(
        filename,
        vertices=vertices,
        faces=faces,
        trans=trans,
        angle=angle,
        camera_focal=camera_info['focal'],
        camera_center=camera_info['center'],
        camera_dist=camera_info['camera_d'],
    )

    print(f"Saved full scene to {filename}")

save_reconstruction_data("HRN_recon.npz", model, result, coeffs, opt, INPUT_IMG_PATH)

In [ ]:
def inspect_saved_file(filename):
    data = np.load(filename)

    print("--- File Contents ---")
    print(f"Vertices: {data['vertices'].shape}")
    print(f"Faces:    {data['faces'].shape}")
    print(f"Position: {data['trans']}")
    print(f"Rotation: {data['angle']}")
    print(f"C_Focal:  {data['camera_focal']}")
    print(f"C_Center: {data['camera_center']}")
    print(f"C_Dist:   {data['camera_dist']}")

inspect_saved_file("HRN_recon.npz")

In [ ]:
from datetime import datetime
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
output_filename = f"TryGlasses_{timestamp}.zip"

!zip {output_filename} \
  HRN_export/HRN_result.obj \
  HRN_export/HRN_result.jpg \
  HRN_export/HRN_preprocessed.jpg \
  HRN_recon.npz

In [ ]:
from google.colab import files
files.download(output_filename)